# Building Data Lake for demographic analysis and prediction in the U.S.

## Data Engineering Capstone Project

### Project Summary
Main goal of the project is to building Data Lake for demographic analysis and prediction in the U.S.

There are several datasets with raw data:

* `I94 Immigration Data`;
* `U.S. City Demographic Data`;
* `Airport Codes`.

Project focuses on creating data model for the future analysis and building ETL-pipeline using Apache Spark. The whole ETL-pipeline processes raw data, cleanup, apply data quality checks and store to the analytical area in the Data Lake according to data model.
Resulting database allows analytics and data scientists to find and predict correlation between demographic situation in different cities/states and immigration to this locations.

The project follows the follow steps:

* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up11

In [5]:
# Do all imports and installs here
import re
import pandas as pd
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

In [2]:
# Set pandas options
pd.set_option('display.width', 500)
pd.set_option('display.max_colwidth', 500)

In [3]:
# Create and configure Spark session. Add support for sas7bdat data format.
spark = (
    SparkSession
    .builder
    .config('spark.jars.packages','saurfang:spark-sas7bdat:2.0.0-s_2.11')
    .config('spark.sql.session.timeZone', 'UTC')
    .enableHiveSupport()
    .getOrCreate()
)

### Step 1: Scope the Project and Gather Data

#### Scope

We will build a database (in the analytical area of our data lake) for demographic analysis and predictions which depends on immegration data.
Our resulting database will have the following structure:

* `airports` – dimension table which contains data about U.S. international airports. This table filled from the raw datasets represented by `I94 Immigration Data` SAS database joined with [airport-codes_csv.csv](./airport-codes_csv.csv) data source files. This table is not necessary for the solution, but `I94 Immigration Data` has own airport identifiers and we need to provide standard airport [IATA](https://en.wikipedia.org/wiki/IATA_airport_code) codes for our analysis team.
* `cities` – dimension table with U.S. cities that will interested us in the future analytics. This table filled from the raw dataset represented by [us-cities-demographics.json](./us-cities-demographics.json) datasource file. We want to analyze only data for the same cities to compare with aggregated data for the 2015 year.
* `immigration` – fact table which contains aggregated data grouped by year, month, and city. Each row of this table shows how much people (men, women and total) immgegrated to the particular U.S. city each month. This table filled from the raw dataset represented by `I94 Immigration Data` SAS database joined with `cities` data.
* `demographics` – pre-aggregated materialized view which is good candidate to land in data mart and contains aggregated data grouped by year and city. It can be used for the fast analytics and reporting. This table filled in two steps:
  1. Filled from the `us-cities-demographics.csv` datasource file which already contains data for the 2015 year.
  2. Filled from the `immigration` fact table by additition to the data for the previous years which allows us to see growth of the population.

##### Important notes and assumptions

1. We assume that all immigrants from the `I94 Immigration Data` will come and stay in the U.S. Actually we could analyze their visa types to be more precises, but for the scope of this project we will not consider it.
2. We assume that all immigrants will stay for living in the city to which the airport belongs. In real life this may not be the case, but we will not consider it.
3. Actually `demographics` table will contains data only for the 2015 and 2016 years because the `I94 Immigration Data` dataset contains data only for the 2016 year but our pipeliens will work for datasets with following years too.

##### Tools

We will use [Apache Spark](https://spark.apache.org/) to build our pipeline. Also we will use Data Lake idea to build our resulting tables and store our data in the HDFS or object storage like S3. We will not create staging area because in the Data Lake approache we can just load raw datasets with Spark, make all necessary transformations, cleanup the data, apply data quality checks and land it in the analytical area.

#### Describe and Gather Data

We will use several data sources to achieve our goal.

1. We will use `U.S. City Demographic Data` grouped by city for 2015 year provided by [U.S. Census Bureau](https://public.opendatasoft.com/explore/dataset/us-cities-demographics/information/).
  * Data format: [JSON](https://en.wikipedia.org/wiki/JSON). You may see both JSON and CSV files with the same dataset in the repository, but we choose JSON format because we want to use different formats for each dataset.
  * Data source: [us-cities-demographics.json](./us-cities-demographics.json)
2. We will extract U.S. immigration data grouped by year, month and city from `I94 Immigration Data` provided by [US National Tourism and Trade Office](https://travel.trade.gov/research/reports/i94/historical/2016.html).
  * Data format: [SAS7BDAT](https://cran.r-project.org/web/packages/sas7bdat/vignettes/sas7bdat.pdf)
  * Data format description: SAS labels and possible values and errors described in the [I94_SAS_Labels_Descriptions.SAS](./I94_SAS_Labels_Descriptions.SAS) file.
  * Data source: [i94_apr16_sub.sas7bdat](../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat). Actually there are bunch of files partitioned by month.
3. Also we will use [Airport Codes](https://datahub.io/core/airport-codes#data) table.
  * Data format: [CSV](https://en.wikipedia.org/wiki/Comma-separated_values)
  * Data source: [airport-codes_csv.csv](./airport-codes_csv.csv)

#### U.S. City Demographic Data

Extract raw data from the data source and display few rows.

As you may see in the example below demogrphic data stored in the `fields` column. There are several fields which we have to extract to achieve our goal:

* city;
* state;
* state_code;
* male_population;
* female_population;
* total_population;

In [6]:
# Load raw data from the U.S. City Demographic Data and display first 5 rows
demographics_source_file_name = './us-cities-demographics.json'
demographics_demo_df = pd.read_json(demographics_source_file_name)
demographics_demo_df.head()

,datasetid,fields,record_timestamp,recordid
0,us-cities-demographics,"{'count': 2177650, 'city': 'Los Angeles', 'number_of_veterans': 85417, 'male_population': 1958998, 'foreign_born': 1485425, 'average_household_size': 2.86, 'median_age': 35.0, 'state': 'California', 'race': 'White', 'total_population': 3971896, 'state_code': 'CA', 'female_population': 2012898}",1970-01-01T03:00:00+03:00,7da42fda61238faccac3d43954a8f621a3a51194
1,us-cities-demographics,"{'count': 124270, 'city': 'Metairie', 'number_of_veterans': 7187, 'male_population': 69515, 'foreign_born': 19871, 'average_household_size': 2.39, 'median_age': 41.6, 'state': 'Louisiana', 'race': 'White', 'total_population': 146458, 'state_code': 'LA', 'female_population': 76943}",1970-01-01T03:00:00+03:00,f176fd28e69ee0c152db080858781e769840c6cc
2,us-cities-demographics,"{'count': 80781, 'city': 'Boca Raton', 'number_of_veterans': 4367, 'male_population': 44760, 'foreign_born': 21117, 'average_household_size': 2.22, 'median_age': 47.3, 'state': 'Florida', 'race': 'White', 'total_population': 93226, 'state_code': 'FL', 'female_population': 48466}",1970-01-01T03:00:00+03:00,ed4a94bf11b553ed8ebd93b7c24b0e8f326dadb0
3,us-cities-demographics,"{'count': 2566, 'city': 'Quincy', 'number_of_veterans': 4147, 'male_population': 44129, 'foreign_born': 32935, 'average_household_size': 2.39, 'median_age': 41.0, 'state': 'Massachusetts', 'race': 'Hispanic or Latino', 'total_population': 93629, 'state_code': 'MA', 'female_population': 49500}",1970-01-01T03:00:00+03:00,a4cd5f6782c79aa0348652718ca179ca390ce086
4,us-cities-demographics,"{'count': 16845, 'city': 'Union City', 'number_of_veterans': 1440, 'male_population': 38599, 'foreign_born': 32752, 'average_household_size': 3.46, 'median_age': 38.5, 'state': 'California', 'race': 'White', 'total_population': 74510, 'state_code': 'CA', 'female_population': 35911}",1970-01-01T03:00:00+03:00,b3b36950de605af92a78168f5127dff485a59b0b


#### I94 Immigration Data

Extract raw data from the data source and display few rows.

There are several fields which we have to extract to achieve our goal:

* `cicid` – unique row identifier;
* `i94yr` – year in 4 digit format;
* `i94mon` – month index (Jan = 1, Feb = 2, Mar = 3, etc.);
* `i94port` – airport code in 3 letters format described in the SAS file description;
* `i94mode` – border crossing method (Air = 1, Sea = 2, Land = 3, Not reported = 9);
* `i94addr` – U.S. state of arrival;
* `gender` – male (M) or female (F).

In [7]:
# Load raw data from the I94 Immigration Data and display first 5 rows
immigration_source_file_name = '../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat'
immigration_demo_df = pd.read_sas(immigration_source_file_name, 'sas7bdat', encoding='ISO-8859-1')
immigration_demo_df.head()

,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,...,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
0,6.0,2016.0,4.0,692.0,692.0,XXX,20573.0,NaN,NaN,NaN,...,U,NaN,1979.0,10282016,NaN,NaN,NaN,1.897628e+09,NaN,B2
1,7.0,2016.0,4.0,254.0,276.0,ATL,20551.0,1.0,AL,NaN,...,Y,NaN,1991.0,D/S,M,NaN,NaN,3.736796e+09,00296,F1
2,15.0,2016.0,4.0,101.0,101.0,WAS,20545.0,1.0,MI,20691.0,...,NaN,M,1961.0,09302016,M,NaN,OS,6.666432e+08,93,B2
3,16.0,2016.0,4.0,101.0,101.0,NYC,20545.0,1.0,MA,20567.0,...,NaN,M,1988.0,09302016,NaN,NaN,AA,9.246846e+10,00199,B2
4,17.0,2016.0,4.0,101.0,101.0,NYC,20545.0,1.0,MA,20567.0,...,NaN,M,2012.0,09302016,NaN,NaN,AA,9.246846e+10,00199,B2


#### Airport Codes

Extract raw data from the data source and display few rows.

There are several fields which we have to extract to achieve our goal:

* `iata_code` – unique [IATA](https://en.wikipedia.org/wiki/IATA_airport_code) airport code, we will extract only international airports and can use this code as identifier;
* `name` – airport name;
* `iso_country` – country, we will use this field to extract only U.S. airports;
* `iso_region` – country + state, we will use it to extract state;
* `municipality` – city to which the airport belongs.

In [12]:
# Load raw data from the Airport Codes and display first 5 rows
airports_source_file_name = './airport-codes_csv.csv'
airports_demo_df = pd.read_csv(airports_source_file_name, sep=',')
airports_demo_df.head()

,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,coordinates
0,00A,heliport,Total Rf Heliport,11.0,NaN,US,US-PA,Bensalem,00A,NaN,00A,"-74.93360137939453, 40.07080078125"
1,00AA,small_airport,Aero B Ranch Airport,3435.0,NaN,US,US-KS,Leoti,00AA,NaN,00AA,"-101.473911, 38.704022"
2,00AK,small_airport,Lowell Field,450.0,NaN,US,US-AK,Anchor Point,00AK,NaN,00AK,"-151.695999146, 59.94919968"
3,00AL,small_airport,Epps Airpark,820.0,NaN,US,US-AL,Harvest,00AL,NaN,00AL,"-86.77030181884766, 34.86479949951172"
4,00AR,closed,Newport Hospital & Clinic Heliport,237.0,NaN,US,US-AR,Newport,NaN,NaN,NaN,"-91.254898, 35.6087"


### Step 2: Explore and Assess the Data

#### Explore the Data

For each of our data sources we will provide it's own cleaning steps.
Important part of this step that we just cleanup and filter raw data sets, we do not perform any joins between our data frames yet. It is preparation step to build our data model in following steps.

#### Cleaning Steps

1. Load and cleanup `U.S. City Demographic Data`
2. Load and cleanup `I94 Immigration Data`
  * Parse `I94_SAS_Labels_Descriptions.SAS`
  * Cleaup `I94 Immigration Data`
3. Load and cleanup `Airport Codes`

#### Load and cleanup U.S. City Demographic Data

Fortunately U.S. City Demographic dataset is pretty good and do not require any cleaning steps. But the essential data lives in the nested object, so we need extract it.


In [41]:
def load_and_cleanup_demographic(source_filename):
    """
        Loads demographic data from the source file in JSON format
        and extract data from the inner JSON property.
        It also should apply cleanup steps, but the existing dataset
        already good quality.
    """

    # Load raw dataset as JSON using Spark
    raw_df = (
        spark
        .read
        .format('json')
        .load(source_filename)
    )

    # Actual data lives in the `fields` property, so we have to extract it.
    cleaned_df = raw_df.select('fields.*')
    return cleaned_df


# Uncomment following code to test load_and_cleanup_demographic function
# demographic_df = load_and_cleanup_demographic('./us-cities-demographics.json')
# demographic_df.show(5)

+----------------------+-----------+-------+-----------------+------------+---------------+----------+------------------+------------------+-------------+----------+----------------+
|average_household_size|       city|  count|female_population|foreign_born|male_population|median_age|number_of_veterans|              race|        state|state_code|total_population|
+----------------------+-----------+-------+-----------------+------------+---------------+----------+------------------+------------------+-------------+----------+----------------+
|                  2.86|Los Angeles|2177650|          2012898|     1485425|        1958998|      35.0|             85417|             White|   California|        CA|         3971896|
|                  2.39|   Metairie| 124270|            76943|       19871|          69515|      41.6|              7187|             White|    Louisiana|        LA|          146458|
|                  2.22| Boca Raton|  80781|            48466|       21117|          

#### Load and cleanup I94 Immigration Data

1. We have a lot of work to cleanup `I94 Immigration Data`. It is stored in SAS format and there is also exists `I94_SAS_Labels_Descriptions.SAS` file with description how to parse dataset values.
We need to write helper function which can map dataset columns values to real values using this description. This is not a cleanup step, but it is important preparation step for the future work.

2. Also we have to cleanup entire `I94 Immigration Data` dataset:

  * `i94port` should be in the list of valid airports described in the `I94_SAS_Labels_Descriptions.SAS` file;
  * `i94mode` = 1, we are interested in only in people who arrived to the U.S. by air, because only for them we can determine destination city;
  * `gender` is not empty, because we want to calculate statistics by men and women separately.

In [22]:
class I94Airport:
    """
        I94Airport contains parsed information from `i94port` field
        of the `I94 Immigration Data`.
        Also it implements static method `sas_description_parse_airport_codes`
        which parses SAS descriptions and returns dictionary
        with airport codes as keys and I94Aiport objects as values.
    """

    def __init__(self, airport_code, us_city, us_state):
        self._airport_code = airport_code
        self._us_city = us_city
        self._us_state = us_state

    def __str__(self):
        return f'{self._us_city}, {self._us_state} [{self._airport_code}]'

    @property
    def airport_code(self):
        return self._airport_code

    @property
    def us_city(self):
        return self._us_city

    @property
    def us_state(self):
        return self._us_state

    @staticmethod
    def sas_description_parse_airport_codes(sas_description_filename):
        """
            Load SAS description file, parse `i94port` values,
            filter non-U.S., "Collapsed" or "No port" data
            and returns dictionary with parsed results to fast lookups.
        """

        airports = {}
        is_airports_section = False

        # We will use regex to extract all three parts (code, city and state) from the description file.
        # Example: 'LOS' = 'LOS ANGELES, CA       '
        # Regex pattern matching: LOS = match[0], LOS ANGELES = match[1], CA = match[2]
        # Also current structure of regex eliminates all invalid airport codes.
        airport_regex = re.compile(r'\'([A-Z0-9]{3})\'.*=.*\'([A-Z-\/,\.\ ]+),\ ?([A-Z]{2})\ *\'')

        with open(sas_description_filename) as f:
            for line in f:
                line = line.strip()
                if line == ';':
                    is_airports_section = False
                elif line == 'value $i94prtl':
                    is_airports_section = True
                elif is_airports_section:
                    airport_parts = airport_regex.findall(line)
                    if airport_parts:
                        airport_code, us_city, us_state = airport_parts[0]
                        airports[airport_code] = I94Airport(airport_code, us_city, us_state)
        return airports


# Uncomment following code to test class I94Airport and it's static method sas_description_parse_airport_codes
# i94_airports = I94Airport.sas_description_parse_airport_codes('./I94_SAS_Labels_Descriptions.SAS')
# i94_airports_df = pd.DataFrame.from_dict(i94_airports, orient='index', columns=['I94 Airport'])
# i94_airports_df.head()

,I94 Airport
ALC,"ALCAN, AK [ALC]"
ANC,"ANCHORAGE, AK [ANC]"
BAR,"BAKER AAF - BAKER ISLAND, AK [BAR]"
DAC,"DALTONS CACHE, AK [DAC]"
PIZ,"DEW STATION PT LAY DEW, AK [PIZ]"


In [19]:
def load_and_cleanup_i94_immigration(sas_source_filename, sas_description_filename):
    """
        Loads and cleanup I94 Immigration Data.
        Apply cleanup steps:
        - i94port should be in the list of valid airports;
        - i94mode = 1 (arrived to the U.S. by air);
        - gender is not empty.
    """

    # Load airports from SAS description file
    i94_airports = I94Airport.sas_description_parse_airport_codes(sas_description_filename)

    # Load raw dataset as SAS using Spark
    raw_df = (
        spark
        .read
        .format('com.github.saurfang.sas.spark')
        .load(sas_source_filename)
    )

    # Cleanup data
    cleaned_df = (
        raw_df
        .filter(F.col('i94mode') == 1)
        .filter(F.col('gender').isNotNull())
        .filter(F.col('i94port').isin(list(i94_airports.keys())))
    )
    return cleaned_df


# Uncomment following code to test load_and_cleanup_i94_immigration function
# immigration_df = load_and_cleanup_i94_immigration('../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat', './I94_SAS_Labels_Descriptions.SAS')
# immigration_df.show(5)

+-----+------+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-----+-------+-------+-------+-------+-------+--------+------+------+-------+--------------+-----+--------+
|cicid| i94yr|i94mon|i94cit|i94res|i94port|arrdate|i94mode|i94addr|depdate|i94bir|i94visa|count|dtadfile|visapost|occup|entdepa|entdepd|entdepu|matflag|biryear| dtaddto|gender|insnum|airline|        admnum|fltno|visatype|
+-----+------+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-----+-------+-------+-------+-------+-------+--------+------+------+-------+--------------+-----+--------+
|  7.0|2016.0|   4.0| 254.0| 276.0|    ATL|20551.0|    1.0|     AL|   null|  25.0|    3.0|  1.0|20130811|     SEO| null|      G|   null|      Y|   null| 1991.0|     D/S|     M|  null|   null|  3.73679633E9|00296|      F1|
| 15.0|2016.0|   4.0| 101.0| 101.0|    WAS|20545.0|    1.0|     MI|20691.0|  55.0|    2.0|  1.0|20160401|    nul

#### Load and cleanup Airport Codes

We are interested only in the U.S. airports, thus we will preserve data by a condition `iso_country` = `US`.
Also we need only international airports, thus we will filter out all data with empty `iata_code`.

In [42]:
def load_and_cleanup_airport_codes(source_filename):
    """
        Loads airport codes data from the source file in CSV format.
        Apply cleanup steps:
        - preserve only U.S. airport codes;
        - remove all rows with empty IATA code.
    """

    # Load raw dataset as CSV using Spark
    raw_df = (
        spark
        .read
        .format('csv')
        .option('header', 'true')
        .option('delimiter', ',')
        .load(source_filename)
    )

    # Cleanup data
    cleaned_df = (
        raw_df
        .filter(F.col('iso_country') == 'US')
        .filter(F.col('iata_code').isNotNull())
    )
    return cleaned_df


# Uncomment following code to test load_and_cleanup_demographic function
# airport_codes_df = load_and_cleanup_airport_codes('./airport-codes_csv.csv')
# airport_codes_df.show(5)

+-----+-------------+--------------------+------------+---------+-----------+----------+-------------+--------+---------+----------+--------------------+
|ident|         type|                name|elevation_ft|continent|iso_country|iso_region| municipality|gps_code|iata_code|local_code|         coordinates|
+-----+-------------+--------------------+------------+---------+-----------+----------+-------------+--------+---------+----------+--------------------+
| 07FA|small_airport|Ocean Reef Club A...|           8|       NA|         US|     US-FL|    Key Largo|    07FA|      OCA|      07FA|-80.274803161621,...|
|  0AK|small_airport|Pilot Station Air...|         305|       NA|         US|     US-AK|Pilot Station|    null|      PQS|       0AK|-162.899994, 61.9...|
| 0CO2|small_airport|Crested Butte Air...|        8980|       NA|         US|     US-CO|Crested Butte|    0CO2|      CSE|      0CO2|-106.928341, 38.8...|
| 0TE7|small_airport|   LBJ Ranch Airport|        1515|       NA|         US

### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
Map out the conceptual data model and explain why you chose that model

#### 3.2 Mapping Out Data Pipelines
List the steps necessary to pipeline the data into the chosen data model

### Step 4: Run Pipelines to Model the Data
#### 4.1 Create the data model
Build the data pipelines to create the data model.

In [ ]:
# Write code here

#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness

Run Quality Checks

In [ ]:
# Perform quality checks here

#### 4.3 Data dictionary
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file.

In [ ]:
#### Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.
* Propose how often the data should be updated and why.
* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
 * The database needed to be accessed by 100+ people.